# Сравниваем исходные картинки

Не забываем убрать нормализацию

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
from nvidia.dali.plugin.pytorch import DALIGenericIterator
import torch
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

from src.dali import CustomPipeline
from src.pytorch import TorchDataset, CV2Dataset

In [ ]:
BATCH_SIZE = 1
IDX = 0

In [ ]:
pipeline = CustomPipeline(batch_size=BATCH_SIZE, num_threads = 8, device_id = 0)
dali_iterator = DALIGenericIterator(pipeline, ['image', 'labels'], size=68811-7)

In [ ]:
torch_dataset = TorchDataset()
torch_dataloader = DataLoader(torch_dataset, batch_size=BATCH_SIZE, num_workers=8, pin_memory=True)

cv2_dataset = CV2Dataset()
cv2_dataloader = DataLoader(cv2_dataset, batch_size=BATCH_SIZE, num_workers=8, pin_memory=True)

In [ ]:
for dali_data in dali_iterator:
    break

In [ ]:
for torch_data in torch_dataloader:
    break

In [ ]:
for cv2_data in cv2_dataloader:
    break

In [ ]:
# Сравниваем размеры батчей
print(f'Размер батча в dali:       {dali_data[0]["image"].cpu().numpy().shape}')
print(f'Размер батча в torchvison: {torch_data["image"].cpu().numpy().shape}')
print(f'Размер батча в cv2:        {cv2_data["image"].cpu().numpy().shape}')

In [ ]:
# Посмотрим на разницу картинок
print('dali vs. torchvision:', torch.abs(dali_data[0]['image'].cpu().float() - torch_data['image'].float()).max().item())
print('dali vs. cv2        :', torch.abs(dali_data[0]['image'].cpu().float() - cv2_data['image'].float()).max().item())
print('cv2 vs. torchvision :', torch.abs(cv2_data['image'].float() - torch_data['image'].float()).max().item())

In [ ]:
# Может картинки не те?

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))

_ = ax1.imshow(dali_data[0]['image'].cpu().numpy()[IDX])
_ = ax1.axis('off')

_ = ax2.imshow(torch_data['image'].numpy()[IDX])
_ = ax2.axis('off')

_ = ax3.imshow(cv2_data['image'].numpy()[IDX])
_ = ax3.axis('off')

In [ ]:
# Визуализируем разницу dali vs torchvision
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
print(
    'Среднее абсолютное отклонение в абсолютных значениях',
    np.abs((dali_data[0]['image'].cpu().float() - torch_data['image'].float()).numpy()).mean(),
)

_ = ax1.imshow((dali_data[0]['image'].cpu().float() - torch_data['image'].float()).numpy()[IDX][..., 0]/255)
_ = ax1.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax1)

_ = ax2.imshow((dali_data[0]['image'].cpu().float() - torch_data['image'].float()).numpy()[IDX][..., 1]/255)
_ = ax2.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax2)

_ = ax3.imshow((dali_data[0]['image'].cpu().float() - torch_data['image'].float()).numpy()[IDX][..., 2]/255)
_ = ax3.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax3)

In [ ]:
# Визуализируем разницу dali vs cv2
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))

print(
    'Среднее абсолютное отклонение в абсолютных значениях',
    np.abs((dali_data[0]['image'].cpu().float() - cv2_data['image'].float()).numpy()).mean(),
)

_ = ax1.imshow((dali_data[0]['image'].cpu().float() - cv2_data['image'].float()).numpy()[IDX][..., 0]/255)
_ = ax1.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax1)

_ = ax2.imshow((dali_data[0]['image'].cpu().float() - cv2_data['image'].float()).numpy()[IDX][..., 1]/255)
_ = ax2.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax2)

_ = ax3.imshow((dali_data[0]['image'].cpu().float() - cv2_data['image'].float()).numpy()[IDX][..., 2]/255)
_ = ax3.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax3)

In [ ]:
# Визуализируем разницу torchvision vs cv2
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
print(
    'Среднее абсолютное отклонение в абсолютных значениях',
    np.abs((torch_data['image'].float() - cv2_data['image'].float()).numpy()).mean(),
)

_ = ax1.imshow((torch_data['image'].float() - cv2_data['image'].float()).numpy()[IDX][..., 0]/255)
_ = ax1.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax1)

_ = ax2.imshow((torch_data['image'].float() - cv2_data['image'].float()).numpy()[IDX][..., 1]/255)
_ = ax2.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax2)

_ = ax3.imshow((torch_data['image'].float() - cv2_data['image'].float()).numpy()[IDX][..., 2]/255)
_ = ax3.axis('off')
_ = f.colorbar(ScalarMappable(Normalize(-1, 1)), ax=ax3)